In [1]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=1e27e318e45ff990249541e1efab95547e5d19f8188d99db7f2b16d7d094a679
  Stored in directory: c:\users\tkhor\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
# !pip install joblib

In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read the CSV and Perform Basic Data Cleaning

In [22]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [23]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
                        'koi_period','koi_period_err1','koi_period_err2',
                        'koi_time0bk','koi_time0bk_err1','koi_time0bk_err2',
                        'koi_impact','koi_impact_err1','koi_impact_err2',
                        'koi_duration','koi_duration_err1','koi_duration_err2',
                        'koi_depth','koi_depth_err1','koi_depth_err2',
                        'koi_prad','koi_prad_err1','koi_prad_err2',
                        'koi_teq','koi_insol','koi_insol_err1','koi_insol_err2',
                        'koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2',
                        'koi_slogg','koi_slogg_err1','koi_slogg_err2',
                        'koi_srad','koi_srad_err1','koi_srad_err2',
                        'ra','dec','koi_kepmag']]
selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [24]:
y = df["koi_disposition"]
X = selected_features
y

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,...,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,...,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,...,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,...,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,...,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [27]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [28]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [29]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [30]:
print(classifier.score(X_train, y_train), classifier.score(X_test, y_test))

0.6522983024985696 0.6418764302059496


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [32]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'penalty': ["l2"],
              'C': np.logspace(2, 6, 8)}
grid = GridSearchCV(classifier, param_grid, verbose=3)

In [33]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] C=100.0, penalty=l2 .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................. C=100.0, penalty=l2, score=0.884, total=   1.2s
[CV] C=100.0, penalty=l2 .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV] ................. C=100.0, penalty=l2, score=0.883, total=   1.5s
[CV] C=100.0, penalty=l2 .............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.6s remaining:    0.0s


[CV] ................. C=100.0, penalty=l2, score=0.882, total=   1.1s
[CV] C=100.0, penalty=l2 .............................................
[CV] ................. C=100.0, penalty=l2, score=0.896, total=   1.4s
[CV] C=100.0, penalty=l2 .............................................
[CV] ................. C=100.0, penalty=l2, score=0.875, total=   1.3s
[CV] C=372.7593720314938, penalty=l2 .................................
[CV] ..... C=372.7593720314938, penalty=l2, score=0.886, total=   1.6s
[CV] C=372.7593720314938, penalty=l2 .................................
[CV] ..... C=372.7593720314938, penalty=l2, score=0.885, total=   1.4s
[CV] C=372.7593720314938, penalty=l2 .................................
[CV] ..... C=372.7593720314938, penalty=l2, score=0.878, total=   1.5s
[CV] C=372.7593720314938, penalty=l2 .................................
[CV] ..... C=372.7593720314938, penalty=l2, score=0.899, total=   1.7s
[CV] C=372.7593720314938, penalty=l2 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.0min finished


GridSearchCV(estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': array([1.00000000e+02, 3.72759372e+02, 1.38949549e+03, 5.17947468e+03,
       1.93069773e+04, 7.19685673e+04, 2.68269580e+05, 1.00000000e+06]),
                         'penalty': ['l2']},
             verbose=3)

In [34]:
print(grid.score(X_train_scaled, y_train))
print(grid.score(X_test_scaled, y_test))

0.8893763112721724
0.8993135011441648


# Save the Model

In [35]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'log_reg.sav'
joblib.dump(grid, filename)

['log_reg.sav']

###### 